In [ ]:
import sys
from itertools import combinations
import numpy as np
import scipy.stats as stats
import torch
import pingouin as pg
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"

from plaid_data import plaid_data
PI_dist_V1 = plaid_data['PI_dist_V1']
PI_dist_V2 = plaid_data['PI_dist_V2']
PI_dist_MT = plaid_data['PI_dist_MT']

# Import custom modules
sys.path.append("../")
from models.network_hierarchical_recurrent import NetworkHierarchicalRecurrent
from models.network_feedforward_stacked import NetworkFeedforwardStacked
from virtual_physiology_20x40.VirtualNetworkPhysiology import VirtualPhysiology
from plotting_functions import *

COLORS = ['tab:blue', 'tab:red', 'tab:green', 'tab:gray']

def not_nan (arr):
    return [a for a in arr if not np.isnan(a)]

In [ ]:
MODEL_PATH = ''
VPHYS_PATH = ''
UNITS      = 800

# Load model
model, hyperparameters, loss_history = NetworkHierarchicalRecurrent.load(
    model_path=MODEL_PATH,
    device='cpu',
    plot_loss_history=False
)

# Load previously processed vphys data
vphys = VirtualPhysiology.load(
    data_path=VPHYS_PATH,
    model=model,
    hyperparameters=hyperparameters,
    hidden_units=[UNITS, UNITS, UNITS],
    frame_shape=(20, 40),
    device='cpu'
)

physiology_data_group1 = vphys.data[0]
physiology_data_group2 = vphys.data[1]
physiology_data_group3 = vphys.data[2]

# Global vars
FRAME_SIZE           = hyperparameters["frame_size"]
WARMUP               = hyperparameters["warmup"]
T_STEPS              = 40
TEMPORAL_FREQUENCIES = vphys.temporal_frequencies
SPATIAL_FREQUENCIES  = vphys.spatial_frequencies
ORIENTATIONS         = vphys.orientations
FRAME_SHAPE          = (20, 40)

COLORS = ['tab:blue', 'tab:red', 'tab:green', 'tab:gray']

HEIGHT, WIDTH = FRAME_SHAPE

# Mean plaid tuning curve

In [ ]:
def get_DSI (tuning_curve):
    orient_pref_idx = np.argmax(tuning_curve)
    orient_pref = np.max(tuning_curve)
    
    orient_opp_idx = (orient_pref_idx + len(tuning_curve)//2) % len(tuning_curve)
    orient_opp = tuning_curve[orient_opp_idx]

    DSI = (orient_pref - orient_opp) / (orient_pref + orient_opp)

    return DSI
    
plaid_selective_DSI_all = []
    
for group in vphys.data:
    plaid_selective_units = [u for u in group if u['plaid_pattern_index']>0]
    plaid_selective_DSI = [get_DSI(u['plaid_response']) for u in plaid_selective_units]
    plaid_selective_DSI_all += plaid_selective_DSI
    
    DS_pct = round(len([u for u in plaid_selective_DSI if u>0.3])/len(plaid_selective_DSI) * 100, 2)
    
    print(len(plaid_selective_DSI), '\t', DS_pct, '%', '\t', np.mean(plaid_selective_DSI))
    
print(
    round(len([u for u in plaid_selective_DSI_all if u>0.3])/len(plaid_selective_DSI_all) * 100, 2),
    '\t', 
    np.mean(plaid_selective_DSI_all)
)

# Distribution with and without feedback

In [ ]:
model_nofb, hyperparameters, _ = NetworkHierarchicalRecurrent.load(
    model_path=MODEL_PATH, device='cpu'
)

pct_inactivated = 1
mask_1 = np.random.choice([0,1], size=(800, 800), p=[pct_inactivated,1-pct_inactivated])
mask_2 = np.random.choice([0,1], size=(800, 800), p=[pct_inactivated,1-pct_inactivated])
weights = model.rnn.weight_hh_l0.detach().cpu().numpy().copy()
weights[0:800, 800:1600] = weights[0:800, 800:1600]*mask_1
weights[800:1600, 1600:2400] = weights[800:1600, 1600:2400]*mask_2
model_nofb.rnn.weight_hh_l0 = torch.nn.Parameter(torch.Tensor(weights).to('cpu'))

# Instantiate new VirtualPhysiology object
vphys_nofb = VirtualPhysiology.load(
    data_path=VPHYS_PATH,
    model=model_nofb,
    hyperparameters=hyperparameters,
    frame_shape=(20, 40),
    hidden_units=[800, 800, 800],
    device='cpu'
)

vphys_nofb.get_plaid_pattern_index()

In [ ]:
full = not_nan([u['plaid_pattern_index'] for u in vphys.data[0]])
nofb = not_nan([u['plaid_pattern_index'] for u in vphys_nofb.data[0]])

mn = np.array([
    np.mean(full),
    np.mean(nofb)
])
er = np.array([
    np.std(full)/(len(full)**2),
    np.mean(nofb)/(len(nofb)**2),
])

x = np.array([0, 1])
labels = ['Full model', 'No feedback']
c = [COLORS[0], COLORS[0]]

fig = plt.figure()
plt.bar(x, mn, yerr=er)
plt.xticks(x, labels)
format_plot(fontsize=20)
fig.set_size_inches((4, 4))
plt.show()

fig = plt.figure()
vp = plt.violinplot([full, nofb], x, showmedians=False, showextrema=False)
plt.xticks(x, labels)
for i, pc in enumerate(vp["bodies"], 0):
    pc.set_facecolor(c[i])
    pc.set_edgecolor('black')
    pc.set_alpha(0.8)
plt.hlines(mn, x-0.5, x+0.5, linewidth=2, color='black', zorder=3, alpha=0.8)
plt.ylabel('Plaid pattern index')
format_plot(fontsize=20)
fig.set_size_inches(5, 4)
plt.savefig(f'./figures/plaid_response/nofb_mean_multicompangle.pdf', bbox_inches='tight')
plt.show()  

pg.ttest(full, nofb, paired=True)

In [ ]:
def centre_shift (arr, axis, shift=False):
    arr = np.array(arr)
    
    if not shift:
        max_idx = np.argmax(arr, axis=axis)
        max_idx = stats.mode(max_idx).mode[0]
        shift = int(np.ceil(arr.shape[axis]/2) - max_idx)
    
    return np.roll(arr, shift, axis=axis), shift

group_idx = 0

plaid_ori_list = np.arange(0, 360, 20)
centred_plaid_ori_list = np.arange(-180, 180, 20)
delta_ori_list = [0, 30, 60, 90, 120, 150]

units = [*vphys.data[0], *vphys.data[2]]

c = 0

for vphys_to_use, model_to_use in zip([vphys, vphys_nofb], [model, model_nofb]):

    for unit_i, unit in enumerate(units):   
        if not unit['hidden_unit_index'] in [2079]: # 2079
            continue

        print('Starting unit ', unit['hidden_unit_index'])

        response_curves_list = []

        for delta_ori in delta_ori_list:
            response_curve = []
            for plaid_ori in plaid_ori_list:
                ori_a = plaid_ori - delta_ori//2
                ori_b = plaid_ori + delta_ori//2

                grating_a = vphys_to_use.get_grating_stimuli(unit['preferred_sf'], ori_a, unit['preferred_tf'], 1, 50)
                grating_b = vphys_to_use.get_grating_stimuli(unit['preferred_sf'], ori_b, unit['preferred_tf'], 1, 50)
                plaid = grating_a+grating_b
                #plaid = (plaid-torch.mean(plaid))/torch.std(plaid)

                with torch.no_grad():
                    _, h = model_to_use(plaid)

                mean_res = np.mean(h[0, WARMUP:, unit['hidden_unit_index']].detach().numpy(), axis=0)
                response_curve.append( mean_res )

            if delta_ori == 0:
                shifted_curve, shift = centre_shift(response_curve, axis=0, shift=False)
            else:
                shifted_curve, _ = centre_shift(response_curve, axis=0, shift=shift)

            response_curves_list.append(shifted_curve)

        fig = plt.figure()

        for curve, delta in zip(response_curves_list, delta_ori_list):
            plt.plot(centred_plaid_ori_list, curve, label=delta)
        plt.ylabel('Unit response')
        plt.xlabel('Orientation (°)')
        format_plot(fontsize=20)
        fig.set_size_inches(4, 4)
        plt.gca().get_legend().set_visible(False)
        plt.show()

        c += 1

# Pattern index distributions

In [ ]:
g1_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[0]])
g2_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[1]])
g3_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[2]])


labels = ['$\mathregular{G_1}$', 'V1', '$\mathregular{G_2}$', 'V2', '$\mathregular{G_3}$', 'MT']
all_dists = [g1_dist, PI_dist_V1, g2_dist, PI_dist_V2, g3_dist, PI_dist_MT]
c = [COLORS[0], 'grey', [168/255,199/255,218/255], 'grey', [213/255,227/255,237/255], 'grey']

means = [np.mean(g1_dist), np.mean(PI_dist_V1),  np.mean(g2_dist), np.mean(PI_dist_V2), np.mean(g3_dist), np.mean(PI_dist_MT)]

x = np.array([0, 1, 2.5, 3.5, 5, 6])

fig = plt.figure()
vp = plt.violinplot(all_dists, x, showmedians=False, showextrema=False)
plt.xticks(x, labels)
for i, pc in enumerate(vp["bodies"], 0):
    pc.set_facecolor(c[i])
    pc.set_edgecolor('black')
    pc.set_alpha(0.8)
plt.hlines(means, x-0.5, x+0.5, linewidth=2, color='black', zorder=3, alpha=0.8)
plt.ylabel('Plaid pattern index')
format_plot(fontsize=20)
fig.set_size_inches(8, 4)
plt.show()  

In [ ]:
anova_df = {
    'group': [],
    'plaid_index': []
}
for g_idx, g in enumerate([g1_dist, g2_dist, g3_dist]):
    anova_df['group'] += list(np.ones(len(g))*g_idx)
    anova_df['plaid_index'] += list(g)
display(pg.anova(data=pd.DataFrame(anova_df), dv='plaid_index', between='group'))

anova_df = {
    'group': [],
    'plaid_index': []
}
for g_idx, g in enumerate([PI_dist_V1, PI_dist_V2, PI_dist_MT]):
    anova_df['group'] += list(np.ones(len(g))*g_idx)
    anova_df['plaid_index'] += list(g)
display(pg.anova(data=pd.DataFrame(anova_df), dv='plaid_index', between='group'))

p_vals = []
label_combos = []
for combo in combinations(zip(labels, all_dists), 2): 
    a, b = combo
    a_label, a_dist = a
    b_label, b_dist = b
    
    ttest = stats.ttest_ind(a_dist, b_dist)
    print(a_label, 'vs', b_label, '\t\t', ttest)
    
    p_vals.append(ttest[1])
    label_combos.append(f'{a_label} vs {b_label}')

p_vals_corr = pg.multicomp(p_vals, method='holm')
    
print('\n')
for pair, p_t, p_v in zip(label_combos, p_vals_corr[0], p_vals_corr[1]):
    print(pair, '\t', p_t, '\t', p_v)

print('\n')
print(np.mean(g1_dist), np.mean(g2_dist), np.mean(g3_dist), np.mean(PI_dist_V1), np.mean(PI_dist_V2), np.mean(PI_dist_MT))


# Model comparison

In [ ]:
def not_nan (arr):
    return [a for a in arr if not np.isnan(a)]

TP_model, hyperparameters, loss_history = NetworkHierarchicalRecurrent.load(
    model_path='',
    device='cpu',
    plot_loss_history=False
)
TP_vphys = VirtualPhysiology.load(
    data_path='',
    model=TP_model,
    hyperparameters=hyperparameters,
    hidden_units=[800, 800, 800],
    frame_shape=(20, 40),
    device='cpu'
)

TP_ff_model = NetworkFeedforwardStacked([])
TP_ff_vphys = VirtualPhysiology.load(
    data_path='',
    model=TP_ff_model,
    hyperparameters=TP_ff_model.stacks[0].hyperparameters,
    hidden_units=[800, 800, 800],
    frame_shape=(20, 40),
    device='cpu'
)

AE_model, hyperparameters, loss_history = NetworkHierarchicalRecurrent.load(
    model_path='',
    device='cpu',
    plot_loss_history=False
)
AE_vphys = VirtualPhysiology.load(
    data_path='',
    model=AE_model,
    hyperparameters=hyperparameters,
    hidden_units=[800, 800, 800],
    frame_shape=(20, 40),
    device='cpu'
)


all_model_dists = []

ks_dist = []

for vphys_i, (vphys, color) in enumerate(zip([TP_vphys, TP_ff_vphys, AE_vphys], COLORS)):
    g1_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[0]])
    g2_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[1]])
    g3_dist = not_nan([u['plaid_pattern_index'] for u in vphys.data[2]])

    all_model_dists.append([g1_dist, g2_dist, g3_dist])
    
    labels = ['$\mathregular{G_{1}}$', 'V1', '$\mathregular{G_{2}}$', 'V2', '$\mathregular{G_{3}}$', 'MT']
    all_dists = [g1_dist, PI_dist_V1, g2_dist, PI_dist_V2, g3_dist, PI_dist_MT]
    c = [color, 'grey', color, 'grey', color, 'grey']

    means = [np.mean(g1_dist), np.mean(PI_dist_V1),  np.mean(g2_dist), np.mean(PI_dist_V2), np.mean(g3_dist), np.mean(PI_dist_MT)]

    x = np.array([0, 1, 3, 4, 6, 7])

    fig = plt.figure()
    vp = plt.violinplot(all_dists, x, showmedians=False, showextrema=False)
    plt.xticks(x, labels)
    for i, pc in enumerate(vp["bodies"], 0):
        pc.set_facecolor(c[i])
        pc.set_edgecolor('black')
        pc.set_alpha(0.8)
    plt.hlines(means, x-0.5, x+0.5, linewidth=2, color='black', zorder=3, alpha=0.8)
    if vphys_i == '':
        plt.ylabel('Plaid pattern index')
    format_plot(fontsize=20)
    fig.set_size_inches(4, 4)
    plt.ylim(-12.5, 12.5)
    plt.savefig(f'./figures/model_comparison/B_{["i", "ii", "iii"][vphys_i]}_multicompangle.pdf', bbox_inches='tight')
    plt.show()  
    
    ks_dist.append([
        stats.ks_2samp(g1_dist, PI_dist_V1)[0],
        stats.ks_2samp(g2_dist, PI_dist_V2)[0],
        stats.ks_2samp(g3_dist, PI_dist_MT)[0]
    ])
    
    p_vals = []
    label_combos = []
    for combo in combinations(zip(labels, all_dists), 2): 
        a, b = combo
        a_label, a_dist = a
        b_label, b_dist = b

        ttest = stats.ttest_ind(a_dist, b_dist)
        print(a_label, 'vs', b_label, '\t\t', ttest)

        p_vals.append(ttest[1])
        label_combos.append(f'{a_label} vs {b_label}')

    
x = [0, 1, 2]
fig = plt.figure()
b = plt.bar(x, [np.mean(d) for d in ks_dist], yerr=[np.std(d)/(len(d)**0.5) for d in ks_dist], facecolor='tab:gray')
for b_i, c_i in zip(b, COLORS):
    b_i.set_facecolor(c_i)
plt.xticks(x, ['$\mathregular{TP_{full}}$', '$\mathregular{TP_{FF}}$', 'AE'], rotation=0)
plt.ylabel('KS distance')
format_plot(fontsize=20)
fig.set_size_inches(3, 4)
plt.savefig(f'./figures/model_comparison/B_iv_multicompangle.pdf', bbox_inches='tight')
plt.show()

# Contour plots

In [ ]:
def centre_shift (arr, axis):
    max_idx = np.argmax(arr, axis=axis)
    max_idx = stats.mode(max_idx).mode[0]
    shift = int(np.ceil(arr.shape[axis]/2) - 1 - max_idx)
    
    return np.roll(arr, shift, axis=axis)

a = np.arange(0, 360, 25)
b = np.arange(0, 360, 25)

units = [*vphys.data[0], *vphys.data[2]]

c = 0

for unit_i, unit in enumerate(units):
    if not unit['hidden_unit_index'] in [8, 585]: # 2079
        continue 
        
    print('Starting unit ', unit['hidden_unit_index'])
    contour = np.zeros(shape=(len(a), len(b)))
    for a_idx, a_i in enumerate(a):
        print('\tProcessing orientation', a_i, 'degrees')
        for b_idx, b_i in enumerate(b):
            grating_a = vphys.get_grating_stimuli(unit['preferred_sf'], a_i, unit['preferred_tf'], 1, 50)
            grating_b = vphys.get_grating_stimuli(unit['preferred_sf'], b_i, unit['preferred_tf'], 1, 50)
            
            plaid = grating_a+grating_b
            plaid = (plaid-torch.mean(plaid))/torch.std(plaid)
            
            with torch.no_grad():
                _, h = model(plaid)


            mean_res = np.mean(h[0, WARMUP:, unit['hidden_unit_index']].detach().numpy(), axis=0)
            contour[a_idx, b_idx] = mean_res

    contour = centre_shift(contour, 0)
    contour = centre_shift(contour, 1)
    
    fig = plt.figure()
    plt.contourf(np.arange(-180, 180, 25), np.arange(-180, 180, 25), contour)
    plt.xlabel('B orientation (°)')
    plt.ylabel('A orientation (°)')
    format_plot(fontsize=20)
    fig.set_size_inches(3, 3)
    plt.savefig(f'./figures/plaid_response/c_{"".join(np.repeat("i", c+1))}.pdf', bbox_inches='tight')
    plt.show()
    
    c += 1

In [ ]:
def centre_shift (arr, axis, shift=False):
    arr = np.array(arr)
    
    if not shift:
        max_idx = np.argmax(arr, axis=axis)
        max_idx = stats.mode(max_idx).mode[0]
        shift = int(np.ceil(arr.shape[axis]/2) - max_idx)
    
    return np.roll(arr, shift, axis=axis), shift

group_idx = 0

plaid_ori_list = np.arange(0, 360, 20)
centred_plaid_ori_list = np.arange(-180, 180, 20)
delta_ori_list = [0, 30, 60, 90, 120, 150]

units = [*vphys.data[0], *vphys.data[2]]

c = 0

for unit_i, unit in enumerate(units):   
    if not unit['hidden_unit_index'] in [8, 585]: # 2079
        continue
        
    print('Starting unit ', unit['hidden_unit_index'])
    
    response_curves_list = []
    
    for delta_ori in delta_ori_list:
        response_curve = []
        for plaid_ori in plaid_ori_list:
            ori_a = plaid_ori - delta_ori//2
            ori_b = plaid_ori + delta_ori//2

            grating_a = vphys.get_grating_stimuli(unit['preferred_sf'], ori_a, unit['preferred_tf'], 1, 50)
            grating_b = vphys.get_grating_stimuli(unit['preferred_sf'], ori_b, unit['preferred_tf'], 1, 50)
            plaid = grating_a+grating_b
            
            with torch.no_grad():
                _, h = model(plaid)

            mean_res = np.mean(h[0, WARMUP:, unit['hidden_unit_index']].detach().numpy(), axis=0)
            response_curve.append( mean_res )
            
        if delta_ori == 0:
            shifted_curve, shift = centre_shift(response_curve, axis=0, shift=False)
        else:
            shifted_curve, _ = centre_shift(response_curve, axis=0, shift=shift)
        
        response_curves_list.append(shifted_curve)

    fig = plt.figure()
    
    for curve, delta in zip(response_curves_list, delta_ori_list):
        plt.plot(centred_plaid_ori_list, curve, label=delta)
    plt.ylabel('Unit response')
    plt.xlabel('Orientation (°)')
    format_plot(fontsize=20)
    fig.set_size_inches(3, 3)
    plt.gca().get_legend().set_visible(False)
    plt.savefig(f'./figures/plaid_response/d_{"".join(np.repeat("i", c+1))}.pdf', bbox_inches='tight')
    plt.show()
    
    c += 1